# SNAPI Demo
This notebook showcases very basic functionality of SNAPI, including querying databases, saving/loading files and manipulating Transient objects.

## (1) Creating a Transient object for 2023ixf.
Here, we will create a mostly empty Transient object for 2023ixf. We will add information through queries.

In [ ]:
from snapi import Transient

ixf_transient = Transient(iid="2023ixf")
print(ixf_transient.id)
print(ixf_transient.coordinates)

Next, we'll call TNS, ALeRCE, and ANTARES to add information based on the IAU name.

In [ ]:
from snapi.query_agents import TNSQueryAgent, ALeRCEQueryAgent, ANTARESQueryAgent

tns_query_agent = TNSQueryAgent()
alerce_query_agent = ALeRCEQueryAgent()
antares_query_agent = ANTARESQueryAgent()

for agent in [tns_query_agent, antares_query_agent]:
    query_results, _ = agent.query_transient(ixf_transient)
    for query_result in query_results:
        ixf_transient.ingest_query_info(query_result.to_dict())
    print(ixf_transient.internal_names)

In [ ]:
# see all unique filters
print(len(ixf_transient.photometry))
for lc in ixf_transient.photometry.light_curves:
    print(lc.filter)

In [ ]:
# see all extracted information
print(ixf_transient.coordinates)
print(ixf_transient.redshift)
print(ixf_transient.spec_class)  # TODO: add spec class extraction

In [ ]:
# plot all photometry
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(8, 6))
ixf_transient.photometry.plot(ax)
plt.legend(ncols=4, loc="lower center")
plt.show()

In [ ]:
# can also plot in flux space
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(8, 6))
ixf_transient.photometry.plot(ax, mags=False)
plt.legend(ncols=4, loc="lower center")
plt.show()

In [ ]:
from snapi import Formatter

# can also plot each LC individually
formatter = Formatter()
for lc in ixf_transient.photometry.light_curves:
    fig, ax = plt.subplots(figsize=(6, 4))
    lc.plot(ax, formatter)
    plt.legend()
    plt.show()